# Krishi Setu: Agricultural Agent (GROW Model)

### Upgraded Features:
1. **Weather**: Integration with OpenWeatherMap for current localized data.
2. **Structured Output**: Strictly enforced GROW model (Goal, Reality, Obstacles, Way Forward) for all responses.

In [19]:
import os
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, OpenWeatherMapAPIWrapper
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain.tools import tool

# Load environment variables
load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["OPENWEATHERMAP_API_KEY"] = os.getenv("OPENWEATHERMAP_API_KEY")

print("Environment initialized.")

Environment initialized.


### Cell A: Tools Definition


In [20]:
# 1. Weather Tool
weather = OpenWeatherMapAPIWrapper()

@tool
def get_weather(location: str) -> str:
    """Fetches the current weather information for a specified location."""
    return weather.run(location)

# 2. Tavily Search
tavily_tool = TavilySearch(max_results=3, topic="general")
tavily_tool.description = "Search Tavily for real-time agricultural market prices and trends."

# 3. Wikipedia
wiki_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki_tool = WikipediaQueryRun(api_wrapper=wiki_wrapper)
wiki_tool.description = "Search Wikipedia for detailed crop facts and botanical information."

tools = [get_weather, tavily_tool, wiki_tool]
print(f"Tools loaded: {[t.name for t in tools]}")

Tools loaded: ['get_weather', 'tavily_search', 'wikipedia']


### Cell B: Model Initialization & Selection


In [21]:
# Initialize Models
llm_qwen = init_chat_model("qwen/qwen3-32b", model_provider="groq")
llm_fast = init_chat_model("llama-3.1-8b-instant", model_provider="groq")

active_model = "qwen" # Change as needed
llm = llm_qwen if active_model == "qwen" else llm_fast

checkpointer = InMemorySaver()

# Intelligent Routing & GROW System Prompt
system_prompt = """You are Krishi Setu, an expert agricultural advisor.

### INTELLIGENT ROUTING:
1. If you know the answer from internal knowledge and it's general, answer directly.
2. If the user asks for current weather, use the 'get_weather' tool.
3. If they ask for live prices, current events, or real-time data, use 'tavily_search'.
4. If they need deep historical or botanical facts, use 'wikipedia'.

### OUTPUT FORMATTING:
After receiving data from any tool, process it and provide your final answer formatted strictly using the **GROW model**:
- **Goal**: The farmer's objective.
- **Reality**: Established facts from tools (weather, prices, etc.).
- **Obstacles**: Potential risks identified.
- **Way Forward**: Clear, actionable advice.

Be concise and professional."""

agent = create_agent(
    model=llm, 
    tools=tools, 
    checkpointer=checkpointer,
    system_prompt=system_prompt
)

print(f"Agent initialized with {llm.model_name} and intelligent routing.")

Agent initialized with qwen/qwen3-32b and intelligent routing.


### Cell C: Multi-Tool Chat Test


In [24]:
config = {"configurable": {"thread_id": "krishi_chat_multi"}}
input_message = "What's the weather like in Guwahati right now, and based on that, what are the current market prices for Rice? Also how this weather effect the crops"

print("Starting Multi-Step Stream Loop...\n")

for step in agent.stream(
    {"messages": [HumanMessage(content=input_message)]}, 
    stream_mode="values", 
    config=config
):
    # Print Thought/Reasoning -> Tool Call -> Tool Message -> Final Answer
    step["messages"][-1].pretty_print()
    print("-" * 50)

Starting Multi-Step Stream Loop...

================================ Human Message =================================

What's the weather like in Guwahati right now, and based on that, what are the current market prices for Rice? Also how this weather effect the crops
--------------------------------------------------
================================== Ai Message ==================================

**Goal**: To assess Guwahati’s current weather, rice market prices, and their impact on crops.  
**Reality**:  
- **Weather**: Guwahati has clear skies at **22.8°C**, **36% humidity**, and **no rainfall**.  
- **Rice Prices**:  
  - Central Government (FCANew): **₹43.15/kg**  
  - Regional Variability: **₹45–₹86/kg** (e.g., basmati, ponni rice).  
- **Crop Effects**:  
  - Clear skies and stable temperatures favor rice growth but **dry conditions** could stress crops without irrigation.  
  - Climate change risks (from Wikipedia) include **reduced productivity**, **soil erosion**, and **pest 

In [25]:
input_message = "based on the weathyewr detail and rice prices and the effect you tell what other crops can be grown in this time for maximum profitability in Assam"

print("Starting Multi-Step Stream Loop...\n")

for step in agent.stream(
    {"messages": [HumanMessage(content=input_message)]}, 
    stream_mode="values", 
    config=config
):
    # Print Thought/Reasoning -> Tool Call -> Tool Message -> Final Answer
    step["messages"][-1].pretty_print()
    print("-" * 50)

Starting Multi-Step Stream Loop...

================================ Human Message =================================

based on the weathyewr detail and rice prices and the effect you tell what other crops can be grown in this time for maximum profitability in Assam
--------------------------------------------------
================================== Ai Message ==================================

**Goal**: To identify high-profitability crops suitable for Assam during the current weather (dry, 22.8°C, 36% humidity) in Guwahati.  
**Reality**:  
- **Weather Conditions**: Dry, clear skies with stable temperatures. No rainfall reported.  
- **Rice Prices**: ₹43.15–₹86/kg (regional), but dry conditions may reduce rice yields.  
- **Climatic Risks**: Dry spells may stress water-dependent crops; irrigation is critical.  

**Obstacles**:  
- Limited rainfall poses challenges for moisture-sensitive crops.  
- Market volatility in rice may reduce profitability for rice-focused farmers.  

**Way 